In [30]:
import numpy as np
import scipy as sp
import os 
import sys
import importlib
import bite_detection_utils as bdu
import copy

In [31]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
#importlib.reload(mdu)

In [32]:
annots = mfileu.read_file('data', 'free_annots_steven_processed.pkl')

In [33]:
RF_our ='our'
all_proba = mfileu.read_file('all_proba2', 'all_proba_bite_free_{}.pkl'.format(RF_our))
pct_proba = mfileu.read_file('pct_proba2', 'pct_proba_bite_free_{}.pkl'.format(RF_our))

In [34]:
def get_labels_free(indices, a):
    if len(indices)==0:
        return []
    
    count = len(indices)    
    labels=np.zeros((count, 2))
    
    isBite = True if indices.shape[1]>1 else False
    
    acount = len(a)    
    for i in range(acount):                 
        ix1 = a[i, 0]
        ix2 = a[i, 1]
        if isBite:
            cond = (indices[:,1]>=ix1) & (indices[:,0]<=ix2)
        else:
            cond = (indices[:, 0]>=ix1) & (indices[:, 0]<=ix2)
            
        labels[cond, 0] = a[i, 2]
    
    cond = (labels[:,0]==1) | (labels[:,0]==2)
    labels[cond, 1] = 1
    
    return labels.astype(int)

In [35]:
def get_bites_frames(bite_frame, percentile_proba, percentile_proba_val, off_on=None):
    assert off_on in [None, "offline", "online"]
    assert bite_frame in ["bite", "frame"]    
    assert percentile_proba in ["percentile", "proba"]    
    
    
    ba = mfileu.read_file('data', 'free_data_steven_blank_array.pkl')
    tp, fp = 0, 0
    
    for subj in range(len(ba)):
        for sess in range(len(ba[subj])):
            a = annots[subj][sess]            
            ix_proba = all_proba[subj][sess][:, :2]
            
            if percentile_proba=='percentile':
                cond  = (pct_proba[:, 0]==subj) & (pct_proba[:, 1]==sess) & (pct_proba[:, 2]==percentile_proba_val)
                assert np.sum(cond)==1                        
                proba_th = pct_proba[cond, -2] if off_on=="offline" else pct_proba[cond, -1]
            else:
                proba_th = percentile_proba_val
            
            if bite_frame=="bite":
                indices = bdu.detect_bites(ix_proba, proba_th=proba_th, ix_offset=40) 
                ba[subj][sess] = indices
                    
            else:
                ix = ix_proba[:, 0]+40
                proba = ix_proba[:, 1]            
                indices = ix[proba>=proba_th].reshape((-1, 1))
            
                l = get_labels_free(indices, a)            
                ba[subj][sess] = np.concatenate((indices, l), axis=1).astype(int)                
                
                tp_1 = np.sum(l[:,1]==1)
                fp_1 = len(l) - tp_1
                
                tp += tp_1
                fp += fp_1           
            
    
    res = {"list":ba, "tp":tp, "fp":fp}
    return res

In [36]:
res ={}
for p in range(9900, 10000):
    percentile = p/100    
    print(RF_our, percentile, end=" | ")    
    x = {}
    x["bite_offline"] = get_bites_frames("bite", "percentile", percentile, "offline")
    x["bite_online"] = get_bites_frames("bite", "percentile", percentile, "online")
    #x["frame_offline"] = get_bites_frames("frame", "percentile", percentile, "offline")
    #x["frame_online"] = get_bites_frames("frame", "percentile", percentile, "online")    
    res[percentile] = x 
    
mfileu.write_file('all_bites_frames', 'all_bites_frames_percentile_{}.pkl'.format(RF_our), res)

our 99.0 | our 99.01 | our 99.02 | our 99.03 | our 99.04 | our 99.05 | our 99.06 | our 99.07 | our 99.08 | our 99.09 | our 99.1 | our 99.11 | our 99.12 | our 99.13 | our 99.14 | our 99.15 | our 99.16 | our 99.17 | our 99.18 | our 99.19 | our 99.2 | our 99.21 | our 99.22 | our 99.23 | our 99.24 | our 99.25 | our 99.26 | our 99.27 | our 99.28 | our 99.29 | our 99.3 | our 99.31 | our 99.32 | our 99.33 | our 99.34 | our 99.35 | our 99.36 | our 99.37 | our 99.38 | our 99.39 | our 99.4 | our 99.41 | our 99.42 | our 99.43 | our 99.44 | our 99.45 | our 99.46 | our 99.47 | our 99.48 | our 99.49 | our 99.5 | our 99.51 | our 99.52 | our 99.53 | our 99.54 | our 99.55 | our 99.56 | our 99.57 | our 99.58 | our 99.59 | our 99.6 | our 99.61 | our 99.62 | our 99.63 | our 99.64 | our 99.65 | our 99.66 | our 99.67 | our 99.68 | our 99.69 | our 99.7 | our 99.71 | our 99.72 | our 99.73 | our 99.74 | our 99.75 | our 99.76 | our 99.77 | our 99.78 | our 99.79 | our 99.8 | our 99.81 | our 99.82 | our 99.83 | o

In [37]:
res ={}
for p in range(10, 95, 5):
    proba = p/100    
    print(RF_our, proba)    
    x = {}
    x["bite"] = get_bites_frames("bite", "proba", proba)    
    #x["frame"] = get_bites_frames("frame", "proba", proba)    
    res[proba] = x 
    
mfileu.write_file('all_bites_frames', 'all_bites_frames_proba_{}.pkl'.format(RF_our), res)

our 0.1
our 0.15
our 0.2
our 0.25
our 0.3
our 0.35
our 0.4
our 0.45
our 0.5
our 0.55
our 0.6
our 0.65
our 0.7
our 0.75
our 0.8
our 0.85
our 0.9
